## Setup

In [1]:
import neurol
import neurol.connect_device
import neurol.BCI
import neurol.streams
import neurol.plot

import serial
import pylsl
import numpy as np

import time

In [2]:
close_char = b'0'
open_char = b'1'
arduino_port = 'com5'

In [3]:
def close_hand(arduino_serial):
    arduino_serial.write(close_char)

def open_hand(arduino_serial):
    arduino_serial.write(open_char)

In [4]:
def action(state):
    '''sends commands to arduino to open or close prosthetic hand'''
    if state:
        close_hand(arduino)
        print('hand closed')
    else:
        open_hand(arduino)
        print('hand open')
    
    # sleep for 1 second to hold action
    time.sleep(1)

In [5]:
# connect to arduino via serial port
print('Connecting Arduino')
arduino = serial.Serial(arduino_port, baudrate=9600)

print(arduino.readline()) # "arduino ready!"

Connecting Arduino
b'ready!\r\n'


In [6]:
close_hand(arduino)
arduino.readline()
arduino.readline()

b'1: 170 (170), 2: 25 (25)\r\n'

In [7]:
open_hand(arduino)
arduino.readline()
arduino.readline()

b'1: 45 (45), 2: 135 (135)\r\n'

In [8]:
# arduino.close()

In [9]:
# get EMG signal stream
stream_info = neurol.connect_device.resolve_byprop('type', 'EMG')[0]
inlet = pylsl.StreamInlet(stream_info)
stream  = neurol.streams.lsl_stream(inlet, buffer_length=512)
print(f'connected to stream \'{inlet.info().name()}\'')


connected to stream 'obci_emg'


## Collect data and train model

In [10]:
# neurol.plot.plot(stream, channels={0: 'EMG'})

In [11]:
stream.n_channels

4

In [12]:
def classifier(emg, clb_info):
    '''classify if hand is closed'''
    return np.average(emg[-20:,0]) > 0.5

def classifier2(emg, clb_info):
    '''alternate classifier'''
    percent_above = np.average(emg[-20:0,0] > 0.5)
    return percent_above > 0.5

# TODO: try neurol's AutoMLBCI w/ TD-FD transformer

In [13]:
# make sure hand is open at start
open_hand(arduino)

In [14]:
# # create neurol BCI
# hand_bci = neurol.BCI.generic_BCI(classifier, transformer, action=action)

# # run prosthetic hand BCI
# try:
#     hand_bci.run(stream)
# except KeyboardInterrupt:
#     print('\n\n\nQUIT')

## Define model

In [15]:
epoch_len = int(stream.sampling_rate * 0.5) # 0.5 seconds
print(f'epoch_len: {epoch_len}')

epoch_len: 100


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Input, Dense, MaxPool1D, Flatten

In [17]:
input_ = Input(shape=(epoch_len, 1), name='input_signal')

conv1 = Conv1D(5, 3, strides=1, padding='same', name='conv1')(input_)
conv2 = Conv1D(5, 3, strides=1, padding='same', name='conv2')(conv1)
maxpool1 = MaxPool1D(pool_size=2, padding='valid', name='maxpool1')(conv2)

conv3 = Conv1D(5, 5, strides=1, padding='same', name='conv3')(maxpool1)
feat_vec = Flatten(name='feat_vec')(maxpool1)

output = Dense(2, activation='softmax', name='output')(feat_vec)

emg_model_tf = tf.keras.Model(inputs=input_, outputs=[output])

In [18]:
emg_model_tf.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_signal (InputLayer)    [(None, 100, 1)]          0         
_________________________________________________________________
conv1 (Conv1D)               (None, 100, 5)            20        
_________________________________________________________________
conv2 (Conv1D)               (None, 100, 5)            80        
_________________________________________________________________
maxpool1 (MaxPooling1D)      (None, 50, 5)             0         
_________________________________________________________________
feat_vec (Flatten)           (None, 250)               0         
_________________________________________________________________
output (Dense)               (None, 2)                 502       
Total params: 602
Trainable params: 602
Non-trainable params: 0
_______________________________________________________________

In [19]:
# compile model. sparse_categorical_crossentropy loss used for int labels.
emg_model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [20]:
n_epochs=100
class automl_bci_model:
    def __init__(self, tf_model):
        self.tf_model = tf_model
    
    def predict(self, x):
        return np.argmax(self.tf_model(x), axis=-1)

    def fit(self, X, Y):
        self.tf_model.fit(X, Y, epochs=n_epochs, verbose=False)

In [21]:
emg_model = automl_bci_model(emg_model_tf)

In [22]:
def transformer(buffer):
    '''returns the EMG channel'''
    return buffer[-epoch_len:,0][:, np.newaxis]

## Define AutoML BCI

In [23]:
automl_hand_bci = neurol.BCI.automl_BCI(emg_model, epoch_len=epoch_len, n_states=2, transformer=transformer, action=action)

In [24]:
automl_hand_bci.build_model(stream, 20)

Recording for 20 seconds...
Recording for 20 seconds...
Done! 

Performance on training data: 
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       157
         1.0       1.00      1.00      1.00       156

    accuracy                           1.00       313
   macro avg       1.00      1.00      1.00       313
weighted avg       1.00      1.00      1.00       313

Performance on test data: 
              precision    recall  f1-score   support

         0.0       0.82      0.85      0.84        39
         1.0       0.85      0.82      0.84        40

    accuracy                           0.84        79
   macro avg       0.84      0.84      0.84        79
weighted avg       0.84      0.84      0.84        79



## Run AutoML BCI

In [25]:
# make sure hand is open at start
open_hand(arduino)

In [26]:
# run prosthetic hand BCI
try:
    automl_hand_bci.run(stream)
except KeyboardInterrupt:
    print('\n\n\nQUIT')

hand closed
hand closed
hand closed
hand closed
hand open
hand open
hand closed
hand closed
hand open
hand closed
hand open
hand closed
hand closed
hand closed
hand open
hand closed
hand open
hand closed
hand closed
hand open
hand closed
hand closed
hand closed
hand closed
hand closed
hand closed
hand open
hand open
hand closed
hand closed
hand open
hand open
hand closed
hand closed
hand open
hand closed
hand closed
hand closed
hand open
hand closed
hand closed
hand closed
hand closed
hand closed
hand closed
hand closed
hand closed
hand open
hand closed
hand closed
hand closed
hand closed
hand closed
hand closed
hand open
hand open
hand closed
hand open
hand closed
hand open
hand open
hand closed
hand closed
hand open
hand open
hand open
hand closed
hand closed
hand closed
hand open
hand closed
hand open
hand open
hand open
hand open
hand closed
hand closed
hand closed



QUIT


In [27]:
open_hand(arduino)